# Analytic Plan Generator

This notebook uses artificial intelligence to generate analytic plans.

## Background

As described in TC 3-12.2.4.1, "The Analytic Scheme of Maneuver is the plan to collect and analyze technical data to meet specific information requirements. It identifies what data to analyze, how to analyze it, and why it is being analyzed." The analytic scheme of maneuver, or ASOM, consists of the following components:

* **Priority information requirement**: 
* **Indicator**: 
* **Evidence**: 
* **Data**: 
* **NAI**: 
* **Analytic**:



## Environment Setup

This section sets up the environment. It installs packages necessary to generate the analytic plans, imports modules, initializes helper functions, and finally defines global variables. This section also mounts Google Drive to the runtime and moves into the project folder.

### Install Packages

In [ ]:
!pip install -U -q "google"
!pip install -U -q "google.genai"
!pip install mitreattack-python

### Import Modules

In [ ]:
import os
from google.colab import userdata
from google.colab import drive
import requests
from mitreattack.stix20 import MitreAttackData
import json
import base64
from google import genai
from google.genai import types
import datetime
import time

### Initialize Helper Functions

In [ ]:
def log(message, end="\n", flush = True):
    """
    Logs a message to the console, prepended with the current timestamp
    in ISO 8601 format.
    
    Args:
    message (str): The string message to log.
    """
    global verbose
    
    # Get the current date and time
    current_time = datetime.datetime.now()
    
    # Format the timestamp in ISO 8601 format
    timestamp = current_time.isoformat()
    
    # Construct the final log string using an f-string for clean formatting
    log_string = f"[{timestamp}] {message}"
    
    # Print the log string to the console if logging is turned on (verbose = True)
    if (verbose == True):
        print(log_string, end = end, flush = flush)

In [ ]:
def extract_attack_id_from_stix(stix_obj):
    """Extracts the MITRE ATT&CK ID (e.g., T1548, TA0002) from a STIX object.

    Iterates through the 'external_references' list of a STIX object (like a technique,
    tactic, or software object) and returns the 'external_id' associated with the
    'mitre-attack' source name.

    Args:
        stix_obj (dict): A dictionary representing a STIX object, expected to have
                         an 'external_references' key.

    Returns:
        str | None: The MITRE ATT&CK external ID if found, otherwise None.
    """
    # Ensure 'external_references' exists and is iterable; default to empty list if not found.
    for ref in stix_obj.get("external_references", []):
        # Check if the reference source is specifically 'mitre-attack'.
        if ref.get("source_name") == "mitre-attack":
            # Return the associated external ID.
            return ref.get("external_id")
    # Return None if no 'mitre-attack' external reference was found.
    return None

### Define Global Variables

In [ ]:
# Toggle logging on (verbose = True)/off (verbose = False)
verbose = True
# verbose = False

In [ ]:
# Rate limits: https://ai.google.dev/gemini-api/docs/rate-limits
# Pricing: https://ai.google.dev/gemini-api/docs/pricing
# Usage: https://console.cloud.google.com/apis/api/generativelanguage.googleapis.com/metrics?project=gen-lang-client-0497172401
# Note that this notebook is designed to be run in Google Colab. The line below reads the Gemini API key for AI Studio,
# which is configured in the Secrets tab on the left side of the Colab window.
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")
log("Gemii API key loaded.")

### Mount Google Drive

In [ ]:
# Mount Google Drive and move into the Google AI Studio folder
DRIVE_PATH = "/content/drive"
TECHNIQUES_PATH = "/content/drive/MyDrive/Google AI Studio/techniques"

drive.mount(DRIVE_PATH)
log(f"Google Drive mounted to {DRIVE_PATH}")

os.chdir(TECHNIQUES_PATH)
log(f"Changed directory to {TECHNIQUES_PATH}")

## ATT&CK Matrix Retrieval

This section first retrieves the latest MITRE ATT&CK Matrix for Enterprise from MITRE's GitHub repository. Then, it builds a data structure that stores each technique as they key with it's parent tactic, the technique's description, and detection suggestions as a nested dictionary as the value.

In [ ]:
# Define the local filename for the MITRE ATT&CK data
attack_data_filename = "enterprise-attack.json"
# Define the source URL for the MITRE ATT&CK Enterprise data
attack_data_url = "https://raw.githubusercontent.com/mitre/cti/master/enterprise-attack/enterprise-attack.json"

# Check if the MITRE ATT&CK data file already exists locally.
# This avoids redundant downloads on subsequent runs, saving time and bandwidth.
# It acts as a simple caching mechanism.
if not os.path.exists(attack_data_filename):
    log(f"Local file '{attack_data_filename}' not found.")
    log (f"Downloading MITRE ATT&CK data from {attack_data_url}")
    try:
        # Attempt to download the latest enterprise ATT&CK framework definition from the official MITRE CTI GitHub repository.
        attack_response = requests.get(attack_data_url)
        # Raise an exception if the download request was unsuccessful (e.g., 404 Not Found, 500 Server Error).
        attack_response.raise_for_status()
        # Get the text content of the successful response.
        attack_text = attack_response.text

        # Parse the downloaded JSON text into a Python dictionary/list structure.
        # This allows programmatic access to the ATT&CK data.
        attack_json = json.loads(attack_text)

        # Save the downloaded and parsed data to the local file.
        # Using 'w' mode ensures that if the file somehow existed but os.path.exists returned False (e.g., race condition, though unlikely here),
        # it would be overwritten. Opening with 'utf-8' encoding is generally good practice for JSON.
        # The 'indent=4' argument makes the saved JSON file human-readable (pretty-printed).
        with open(attack_data_filename, "w", encoding='utf-8') as outfile:
            json.dump(attack_json, outfile, indent=4)
        log(f"Successfully downloaded and saved data to '{attack_data_filename}'.")

    except requests.exceptions.RequestException as e:
        # Handle potential network errors during download (e.g., connection error, timeout, bad HTTP status).
        log(f"Error downloading ATT&CK data: {e}")
        # Depending on the application's needs, might want to exit, retry, or proceed without the data.
        # For now, we'll just print the error and potentially fail later when MitreAttackData is initialized.
    except json.JSONDecodeError as e:
        # Handle cases where the downloaded content is not valid JSON.
        log(f"Error parsing downloaded JSON data: {e}")
    except IOError as e:
        # Handle potential errors during file writing (e.g., permissions denied, disk full).
        log(f"Error writing data to file '{attack_data_filename}': {e}")

# Initialize the MitreAttackData object using the local JSON file.
# This object provides methods to easily query and navigate the ATT&CK framework data
# (e.g., find techniques, tactics, mitigations, relationships).
# It encapsulates the logic for interacting with the raw JSON data structure.
# Ensure the file exists before attempting to initialize, or handle potential FileNotFoundError within MitreAttackData.
if os.path.exists(attack_data_filename):
    try:
        mad = MitreAttackData(attack_data_filename)
        log(f"MitreAttackData object initialized successfully from '{attack_data_filename}'.")
        # Now the 'mad' object can be used to work with the ATT&CK data.
        # Example (assuming methods exist):
        # techniques = mad.get_techniques()
        # tactic_details = mad.get_tactic_by_id('TA0001')
    except Exception as e:
        # Catch any potential errors during MitreAttackData initialization (e.g., file format issues not caught earlier, internal errors).
        log(f"Error initializing MitreAttackData from '{attack_data_filename}': {e}")
        # Set mad to None or handle appropriately to indicate failure.
        mad = None
else:
    # This branch is reached if the download/file writing failed in the 'if not os.path.exists' block.
    log(f"Cannot initialize MitreAttackData: File '{attack_data_filename}' is missing and could not be created.")
    sys.exit(1)

Now that the script has retrieved the MITRE ATT&CK Matrix for Enterprise, transform it into a new data structure for ease of reference later.

In [ ]:
# Create a dictionary mapping lowercase tactic names (e.g., "initial access")
# to their corresponding MITRE ATT&CK Tactic IDs (e.g., "TA0001").
# This provides a quick lookup for tactic IDs based on names found in technique data.
# It assumes the first external reference in a tactic object contains the desired ID.
tactic_name_to_id = {
    # Convert tactic name to lowercase for case-insensitive matching later.
    tactic.name.lower(): tactic.external_references[0]["external_id"]
    # Iterate through all tactic objects provided by the MitreAttackData instance.
    for tactic in mad.get_tactics()
}

# Initialize the dictionary that will store the processed technique/sub-technique information.
# The keys will be formatted strings like "T1548.002 - Unsecured Credentials",
# and the values will be dictionaries containing tactic, description, and detection details.
technique_dict = {}

# Retrieve both techniques and sub-techniques from the MitreAttackData instance.
# Exclude any techniques/sub-techniques marked as revoked or deprecated in the source data.
# Combine them into a single list for uniform processing.
all_techniques = mad.get_techniques(remove_revoked_deprecated=True) + \
                 mad.get_subtechniques(remove_revoked_deprecated=True)

# Iterate through each technique or sub-technique object in the combined list.
for tech in all_techniques:
    # Extract the primary ATT&CK ID (e.g., "T1548" or "T1548.002") using the helper function.
    tid = extract_attack_id_from_stix(tech)
    # Skip processing if a valid ATT&CK ID couldn't be extracted.
    if not tid:
        continue # Or add logging/error handling

    # Extract the name, remove leading/trailing whitespace, and replace slashes
    # (which might cause issues in file paths or keys) with hyphens.
    name = tech.get("name", "").strip().replace("/", "-")

    # Create a unique, human-readable key for the technique_dict.
    full_key = f"{tid} - {name}"

    # Extract the description and detection details, removing leading/trailing whitespace.
    # Use .get() with a default empty string to prevent errors if keys are missing.
    description = tech.get("description", "").strip()
    # 'x_mitre_detection' is a custom STIX property used by MITRE for detection guidance.
    detection = tech.get("x_mitre_detection", "").strip()

    tactic_names = []
    # Iterate through the kill chain phases associated with the technique/sub-technique.
    # Default to an empty list if 'kill_chain_phases' is missing.
    for phase in tech.get("kill_chain_phases", []):
        # Ensure we are looking at phases belonging to the 'mitre-attack' framework
        # (as opposed to potential other frameworks like 'mitre-pre-attack').
        if phase.get("kill_chain_name") == "mitre-attack":
            # Extract the tactic's phase name (e.g., "initial-access").
            # Convert it to lowercase and replace hyphens with spaces (e.g., "initial access")
            # to match the keys created for the 'tactic_name_to_id' dictionary.
            phase_name_lookup_key = phase.get("phase_name", "").lower().replace("-", " ")

            # Look up the Tactic ID (e.g., "TA0001") using the prepared lookup key.
            tactic_id = tactic_name_to_id.get(phase_name_lookup_key)

            # If a corresponding tactic ID was found...
            if tactic_id:
                # Format the tactic information string (e.g., "TA0001 - Initial Access").
                # Use .title() for consistent capitalization of the tactic name.
                tactic_display_name = phase.get("phase_name", "").replace("-", " ").title()
                tactic_names.append(f"{tactic_id} - {tactic_display_name}")

    # Join the collected tactic strings into a single comma-separated string.
    # Use set() to remove potential duplicates and sorted() for consistent ordering.
    tactic_str = ", ".join(sorted(set(tactic_names)))
    # --- End Tactic Processing ---

    # Store the extracted and formatted information in the main dictionary.
    technique_dict[full_key] = {
        "tactic": tactic_str,
        "description": description,
        "detection": detection,
    }

In [ ]:
# Debugging block
# 'technique_dict' now contains the processed data, ready for further use
# (e.g., writing to a file, displaying in a UI, further analysis).
# print(json.dumps(technique_dict["T1133 - External Remote Services"], indent=4))

## Build Prompts to Generate Initial Analytic Plans

This section generates a prompt to build the initial analytic plan for each technique in the MITRE ATT&CK matrix. It first defines a base prompt that provides context to the model, and then generates a specific prompt crafted for each individual technique. The complete prompts are stored in `prompt_library`, where the key is the technique ID and name in the form "T1190 - Exploit Public-Facing Application".

In [ ]:
# This prompt generates the initial analytic plan.
base_prompt = """\
Generate an Analytic Scheme of Maneuver (ASOM) based on the following definitions of its components. For each component, provide specific examples relevant to a scenario where we suspect a sophisticated external threat is attempting to gain unauthorized access to our organization's sensitive data or negatively impact the network.

ASOM Component Definitions:

1.  Information Requirement (IR): These identify the information about the enemy or the terrain that the commander considers most important. For example, "Has the adversary gained initial access to the network?" These should be tagged with MITRE ATT&CK tactic numbers; for example, (TA0001 - Initial Access). A complete PIR looks like this: "Has the adversary gained initial access to the network? (TA0001 - Initial Access)".

2.  Indicators: These are positive or negative evidence of threat activity pertaining to one or more information requirements. They are observable clues related to a specific information requirement. For the IR "Has the adversary gained initial access to the network? (TA0001 - Initial Access)", the indicator should be a technique within that tactic; for example, "T1190 - Exploit Public Facing Application".

3.  Evidence: This is the concrete information that supports or refutes an indicator. It provides the "proof" and can vary in complexity. For the IR "Has the adversary gained initial access to the network?" and the indicator "T1190 - Exploit Public Facing Application" beneath it, appropriate evidence could be:
    * "Anomalous login attempts from unusual geographic locations."
    * "Network traffic involving known malicious command and control (C2) infrastructure."

4.  Data: This describes the precise data necessary to identify evidence. Specificity here is key (e.g., Zeek Conn logs, Sysmon event ID 4624, Active Directory security logs). For the evidence, focus your ASOM on the following data sources: network logs, specifically Zeek logs; host logs, specifically Windows Event IDs. Write only the data name. For example, Windows Event ID 4688; Zeek conn.log

5. Data Platform: This describes the type of system from which the data can be collected. It should be one of the following options: "Endpoints", "Servers", "Network devices".

6.  Named Areas of Interest (NAIs): These are areas where data that will satisfy a specific information requirement can be collected. For all NAIs, use a dummy value of "Insert site-specific NAI here".

7.  Actions: These are high-level instructions that guide the analysts' search for evidence. Focus mostly on simple detections, but also look for opportunities to incorporate basic statistical methods data science techniques here, such as descriptive statistics, Inter‑quartile range & box‑plots, entropy measures, correlation analysis, linear regression, time series analysis, and other, similar methods. For the evidence above, appropriate actions could be:
    * "Identify all source IP addresses for failed and successful login events. Geolocate logain sources, then identify rare source countries by low frequency of occurence or percentiles to flag anomalous login events from unusual geographic locations."
    * "Inner join a list of source and destination IP addresses from public-facing devices with a list of known malicious command and control (C2) servers from a cyber threat intelligence provider updated within at least 30 days. Investigate all sessions where connections involved C2 servers."

Based on these definitions, please generate a detailed ASOM in the JSON format. The keys in the JSON object should correspond to the following ASOM components in this order: IR, Indicators, Evidence, Data Platform, Data, NAIs, Actions. Note that the key for IR should be replaced by the actual IR based on the description above. Also note that the key for evidence should be replaced with the actual form of evidence based on the description above. Here is an example template:

{
    "(Insert IR here)": {
        "Indicators": {
            "(Insert form of evidence here)": {
                "Data": "",
                "Data Platform": "",
                "NAI": "",
                "Action": ""
            },
            "(Insert form of evidence here)": {
                "Data": "",
                "Data Platform": "",
                "NAI": "",
                "Action": ""
            }
        }
    }
}

For each MITRE technique below, generate one PIR (a general question tagged with the parent tactic's T-code, in the format "Has the adversary gained initial access to the network? (TA0001 - Initial Access)") for each tactic. For example, if there is one parent tactic, generate one PIR; if there are two parent tactics, generate two PIRs. For each PIR, generate two indicators (the MITRE technique provided). For each indicator, generate one form of evidence. Each form of evidence should have Data, NAI, and Actions. I have also provided a helpful description of the technique labeled "Description:", and a potential idea for detecting it, labeled "Detection:"."""

In [ ]:
prompt_library = {}

for technique in technique_dict:
    prompt_library[technique] = base_prompt + f"\n\nTechnique: {technique}\n\nTactic(s): {technique_dict[technique]['tactic']}\n\nDescription: {technique_dict[technique]['description']}\n\nDetection: {technique_dict[technique]['detection']}"

In [ ]:
# print(prompt_library["T1190 - Exploit Public-Facing Application"])

## Generate the Initial Analytic Plans

This section generates one PIR and two indicators (with evidence, data, data platform, NAI, and action) for each technique in the MITRE ATT&CK matrix for Enterprise. Note that if a technique may be used at multiple points in an operation, as part of more than one tactic, a unique PIR is generated for each instance of that technique.

First, define a function with embeded context for generating the analytic plans. This function is unique to the initial generation of analytic plans.

In [ ]:
def generate_initial_plan(prompt, target_model):
    """Generates an analytic plan in JSON format using a generative AI model.

    This function constructs a detailed, multi-turn prompt including definitions, examples,
    and constraints to guide the target generative model in creating an analytic plan tailored
    to a specific cyber threat scenario and MITRE ATT&CK technique. It leverages
    few-shot prompting by providing a complete example (for T1133) within the prompt context.

    Args:
        prompt (str): A string containing the specific details of the MITRE ATT&CK
                      technique for which the ASOM should be generated. This typically
                      includes Technique ID, Name, Tactics, Description, and Detection guidance.
        target_model (str): The name or identifier of the generative AI model to use
                            (e.g., "models/gemini-1.5-flash").

    Returns:
        str: A string containing the generated ASOM in JSON format. Returns the raw text
             response from the model. Error handling for API calls or invalid JSON
             responses is not explicitly included here.
    """
    # Initialize the Generative AI client using the API key stored in environment variables.
    # Ensure the 'GEMINI_API_KEY' environment variable is set before running.
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    # Set the target model for the generation request.
    model = target_model

    # Construct the conversation history ('contents') for the generative model.
    # This uses a multi-turn approach to provide detailed instructions and examples (few-shot learning).
    contents = [
        # Turn 1: User provides detailed instructions, definitions, constraints, and a full example.
        types.Content(
            role="user",
            parts=[
                # This large text block defines the ASOM structure, components (IR, Indicators, etc.),
                # the desired JSON output format, data source constraints (Zeek, Win Events),
                # NAI placeholder requirements, action guidance (including data science methods),
                # and crucially, provides a complete example scenario based on T1133.
                # --- START OF DETAILED PROMPT TEXT (DO NOT MODIFY CONTENT) ---
                types.Part.from_text(text="""Generate an Analytic Scheme of Maneuver (ASOM) based on the following definitions of its components. For each component, provide specific examples relevant to a scenario where we suspect a sophisticated external threat is attempting to gain unauthorized access to our organization's sensitive data or negatively impact the network.

ASOM Component Definitions:

1.  Information Requirement (IR): These identify the information about the enemy or the terrain that the commander considers most important. For example, \"Has the adversary gained initial access to the network?\" These should be tagged with MITRE ATT&CK tactic numbers; for example, (TA0001 - Initial Access). A complete PIR looks like this: \"Has the adversary gained initial access to the network? (TA0001 - Initial Access)\".

2.  Indicators: These are positive or negative evidence of threat activity pertaining to one or more information requirements. They are observable clues related to a specific information requirement. For the IR \"Has the adversary gained initial access to the network? (TA0001 - Initial Access)\", the indicator should be a technique within that tactic; for example, \"T1190 - Exploit Public Facing Application\".

3.  Evidence: This is the concrete information that supports or refutes an indicator. It provides the \"proof\" and can vary in complexity. For the IR \"Has the adversary gained initial access to the network?\" and the indicator \"T1190 - Exploit Public Facing Application\" beneath it, appropriate evidence could be:
    * \"Anomalous login attempts from unusual geographic locations.\"
    * \"Network traffic involving known malicious command and control (C2) infrastructure.\"

4.  Data: This describes the precise data necessary to identify evidence. Specificity here is key (e.g., Zeek Conn logs, Sysmon event ID 4624, Active Directory security logs). For the evidence, focus your ASOM on the following data sources: network logs, specifically Zeek logs; host logs, specifically Windows Event IDs. Write only the data name. For example, Windows Event ID 4688; Zeek conn.log

5. Data Platform: This describes the type of system from which the data can be collected. It should be one of the following options: \"Endpoints\", \"Servers\", \"Network devices\".

6.  Named Areas of Interest (NAIs): These are areas where data that will satisfy a specific information requirement can be collected. For all NAIs, use a dummy value of \"Insert site-specific NAI here\".

7.  Actions: These are high-level instructions that guide the analysts' search for evidence. Focus mostly on simple detections, but also look for opportunities to incorporate basic statistical methods data science techniques here, such as descriptive statistics, Inter‑quartile range & box‑plots, entropy measures, correlation analysis, linear regression, time series analysis, and other, similar methods. For the evidence above, appropriate actions could be:
    * \"Identify all source IP addresses for failed and successful login events. Geolocate logain sources, then identify rare source countries by low frequency of occurence or percentiles to flag anomalous login events from unusual geographic locations.\"
    * \"Inner join a list of source and destination IP addresses from public-facing devices with a list of known malicious command and control (C2) servers from a cyber threat intelligence provider updated within at least 30 days. Investigate all sessions where connections involved C2 servers.\"

Based on these definitions, please generate a detailed ASOM in the JSON format. The keys in the JSON object should correspond to the following ASOM components in this order: IR, Indicators, Evidence, Data Platform, Data, NAIs, Actions. Note that the key for IR should be replaced by the actual IR based on the description above. Also note that the key for evidence should be replaced with the actual form of evidence based on the description above. Here is an example template:

{
    \"(Insert IR here)\": {
        \"Indicators\": {
            \"(Insert form of evidence here)\": {
                \"Data\": \"\",
                \"Data Platform\": \"\",
                \"NAI\": \"\",
                \"Action\": \"\"
            },
            \"(Insert form of evidence here)\": {
                \"Data\": \"\",
                \"Data Platform\": \"\",
                \"NAI\": \"\",
                \"Action\": \"\"
            }
        }
    }
}

For each MITRE technique below, generate one PIR (a general question tagged with the parent tactic's T-code, in the format \"Has the adversary gained initial access to the network? (TA0001 - Initial Access)\") for each tactic. For example, if there is one parent tactic, generate one PIR; if there are two parent tactics, generate two PIRs. For each PIR, generate two indicators (the MITRE technique provided). For each indicator, generate one form of evidence. Each form of evidence should have Data, NAI, and Actions. I have also provided a helpful description of the technique labeled \"Description:\", and a potential idea for detecting it, labeled \"Detection:\".

Technique: T1133 - External Remote Services

Tactic: TA0001 - Initial-Access, TA0003 - Persistence

Description: Adversaries may leverage external-facing remote services to initially access and/or persist within a network. Remote services such as VPNs, Citrix, and other access mechanisms allow users to connect to internal enterprise network resources from external locations. There are often remote service gateways that manage connections and credential authentication for these services. Services such as [Windows Remote Management](https://attack.mitre.org/techniques/T1021/006) and [VNC](https://attack.mitre.org/techniques/T1021/005) can also be used externally.(Citation: MacOS VNC software for Remote Desktop)

Access to [Valid Accounts](https://attack.mitre.org/techniques/T1078) to use the service is often a requirement, which could be obtained through credential pharming or by obtaining the credentials from users after compromising the enterprise network.(Citation: Volexity Virtual Private Keylogging) Access to remote services may be used as a redundant or persistent access mechanism during an operation.

Access may also be gained through an exposed service that doesn’t require authentication. In containerized environments, this may include an exposed Docker API, Kubernetes API server, kubelet, or web application such as the Kubernetes dashboard.(Citation: Trend Micro Exposed Docker Server)(Citation: Unit 42 Hildegard Malware)

Detection: Follow best practices for detecting adversary use of [Valid Accounts](https://attack.mitre.org/techniques/T1078) for authenticating to remote services. Collect authentication logs and analyze for unusual access patterns, windows of activity, and access outside of normal business hours.

When authentication is not required to access an exposed remote service, monitor for follow-on activities such as anomalous external use of the exposed API or application."""),
                # --- END OF DETAILED PROMPT TEXT ---
            ],
        ),
        # Turn 2: Model provides its internal plan for generating the example (T1133).
        # This helps guide the model's reasoning process for the actual prompt later.
        types.Content(
            role="model",
            parts=[
                # This text simulates the model outlining its step-by-step plan to generate the ASOM for T1133.
                # --- START OF MODEL PLAN TEXT (DO NOT MODIFY CONTENT) ---
                types.Part.from_text(text="""The user wants an Analytic Scheme of Maneuver (ASOM) in JSON format.
The ASOM should follow a specific structure: IR -> Indicators -> Evidence -> Data, Data Platform, NAI, Action.
The IR key should be the actual Information Requirement string.
The Evidence key should be the actual evidence string.
The scenario involves detecting a sophisticated external threat attempting unauthorized access or network impact.
I need to generate PIRs, Indicators, Evidence, Data, Data Platform, NAI, and Actions based on the provided technique T1133 - External Remote Services.

**Plan:**

1.  **Identify Tactics:** T1133 belongs to TA0001 (Initial Access) and TA0003 (Persistence).
2.  **Formulate PIRs:** Create one PIR for each tactic, using the general question format provided and the tactic T-code.
    * PIR 1 (TA0001): \"Has the adversary gained initial access to the network via external remote services? (TA0001 - Initial Access)\"
    * PIR 2 (TA0003): \"Is the adversary maintaining persistence using external remote services? (TA0003 - Persistence)\"
3.  **Define Indicators:** For each PIR, use the provided technique T1133 as the indicator. Since the user asked for *two* indicators per PIR, but only provided *one* technique (T1133), I will use T1133 twice for each PIR, but focus on different aspects or detection methods derived from the description and detection notes.
    * PIR 1 -> Indicator 1: T1133 - External Remote Services (focus on login anomalies)
    * PIR 1 -> Indicator 2: T1133 - External Remote Services (focus on exposed services)
    * PIR 2 -> Indicator 1: T1133 - External Remote Services (focus on persistent unusual access)
    * PIR 2 -> Indicator 2: T1133 - External Remote Services (focus on redundant access mechanisms)
4.  **Develop Evidence:** For each indicator, create a specific form of evidence based on the detection notes.
    * PIR1/Indicator1 (T1133 - Login Anomalies): \"Anomalous login attempts to external remote services (VPN, Citrix, RDP) from unusual geographic locations or outside normal business hours.\"
    * PIR1/Indicator2 (T1133 - Exposed Services): \"Anomalous external usage patterns targeting potentially exposed remote services (e.g., Docker API, Kubernetes API) that do not require authentication.\"
    * PIR2/Indicator1 (T1133 - Persistent Unusual Access): \"Successful logins to external remote services exhibiting unusual access patterns (e.g., multiple failed logins followed by success, logins at odd hours consistently over time) matching known Valid Accounts.\"
    * PIR2/Indicator2 (T1133 - Redundant Access): \"Connections via external remote services originating from IP addresses associated with other known suspicious activities or previously compromised accounts.\"
5.  **Specify Data Sources:** Identify the specific logs needed for each piece of evidence, focusing on Windows Event IDs and Zeek logs as requested.
    * Evidence 1 (Login Anomalies): Windows Event ID 4624, Windows Event ID 4625, VPN Logs, Citrix Logs, Zeek conn.log
    * Evidence 2 (Exposed Services): Zeek conn.log, Web Server Access Logs (if applicable for K8s dashboard etc.)
    * Evidence 3 (Persistent Unusual Access): Windows Event ID 4624, Windows Event ID 4625, VPN Logs, Citrix Logs
    * Evidence 4 (Redundant Access): Zeek conn.log, VPN Logs, Citrix Logs, Threat Intel Feeds
6.  **Define Data Platform:** Assign the appropriate platform (\"Endpoints\", \"Servers\", \"Network devices\").
    * Evidence 1: Servers (for AD/Event Logs), Network devices (VPN/Citrix/Zeek)
    * Evidence 2: Network devices (Zeek), Servers (Web server logs)
    * Evidence 3: Servers (for AD/Event Logs), Network devices (VPN/Citrix)
    * Evidence 4: Network devices (Zeek/VPN/Citrix), Servers (Threat Intel platform)
7.  **Set NAI:** Use the placeholder \"Insert site-specific NAI here\".
8.  **Create Actions:** Define actions to find the evidence, incorporating basic analysis and statistical methods where applicable.
    * Evidence 1 Action: \"Collect remote service authentication logs (Windows Event ID 4624, 4625, VPN/Citrix). Identify source IP addresses for successful and failed logins. Geolocate source IPs. Calculate frequency distribution of source countries/regions and login times. Flag logins from low-frequency locations or occurring outside typical business hours (e.g., using IQR or percentile analysis on login times).\"
    * Evidence 2 Action: \"Monitor Zeek conn.log for connections to known ports associated with exposed services (e.g., Docker API 2375/2376, K8s API 6443/443, kubelet 10250). Analyze connection volume, source IPs, and request patterns. Use time series analysis to detect sudden spikes or changes in traffic patterns from external IPs to these services.\"
    * Evidence 3 Action: \"Correlate successful logins (Event ID 4624, VPN/Citrix logs) with preceding failed attempts (Event ID 4625). Analyze login timestamps for patterns inconsistent with normal user behavior (e.g., logins consistently occurring late at night or on weekends). Use descriptive statistics to establish baseline login behavior per user/group and identify deviations.\"
    * Evidence 4 Action: \"Cross-reference source IP addresses from remote service logs (Zeek conn.log, VPN/Citrix logs) against a list of IP addresses identified in other security alerts or known compromised host lists. Inner join connection logs with Threat Intelligence feeds for known malicious IPs connecting via remote services.\"
9.  **Assemble JSON:** Construct the final JSON object following the specified structure and order. Use the PIRs as the top-level keys. Use the evidence strings as the keys under \"Indicators\"."""),
                # --- END OF MODEL PLAN TEXT ---
                # This provides the expected JSON output for the T1133 example, acting as a strong few-shot example.
                # --- START OF MODEL EXAMPLE JSON OUTPUT (DO NOT MODIFY CONTENT) ---
                types.Part.from_text(text="""[
  {
    \"Has the adversary gained initial access to the network via external remote services? (TA0001 - Initial Access)\": {
      \"Indicators\": {
        \"T1133 - External Remote Services\": {
          \"Anomalous login attempts to external remote services (VPN, Citrix, RDP) from unusual geographic locations or outside normal business hours.\": {
            \"Data\": \"Windows Event ID 4624, Windows Event ID 4625, Zeek conn.log\",
            \"Data Platform\": \"Servers, Network devices\",
            \"NAI\": \"Insert site-specific NAI here\",
            \"Action\": \"Collect remote service authentication logs (Windows Event ID 4624, 4625, VPN/Citrix). Identify source IP addresses for successful and failed logins. Geolocate source IPs. Calculate frequency distribution of source countries/regions and login times. Flag logins from low-frequency locations or occurring outside typical business hours (e.g., using IQR or percentile analysis on login times).\"
          },
          \"Anomalous external usage patterns targeting potentially exposed remote services (e.g., Docker API, Kubernetes API) that do not require authentication.\": {
            \"Data\": \"Zeek conn.log\",
            \"Data Platform\": \"Network devices\",
            \"NAI\": \"Insert site-specific NAI here\",
            \"Action\": \"Monitor Zeek conn.log for connections to known ports associated with exposed services (e.g., Docker API 2375/2376, K8s API 6443/443, kubelet 10250). Analyze connection volume, source IPs, and request patterns. Use time series analysis to detect sudden spikes or changes in traffic patterns from external IPs to these services.\"
          }
        }
      }
    }
  },
  {
    \"Is the adversary maintaining persistence using external remote services? (TA0003 - Persistence)\": {
      \"Indicators\": {
        \"T1133 - External Remote Services\": {
          \"Successful logins to external remote services exhibiting unusual access patterns (e.g., multiple failed logins followed by success, logins at odd hours consistently over time) matching known Valid Accounts.\": {
            \"Data\": \"Windows Event ID 4624, Windows Event ID 4625\",
            \"Data Platform\": \"Servers\",
            \"NAI\": \"Insert site-specific NAI here\",
            \"Action\": \"Correlate successful logins (Event ID 4624, VPN/Citrix logs) with preceding failed attempts (Event ID 4625). Analyze login timestamps for patterns inconsistent with normal user behavior (e.g., logins consistently occurring late at night or on weekends). Use descriptive statistics to establish baseline login behavior per user/group and identify deviations.\"
          },
          \"Connections via external remote services originating from IP addresses associated with other known suspicious activities or previously compromised accounts.\": {
            \"Data\": \"Zeek conn.log\",
            \"Data Platform\": \"Network devices\",
            \"NAI\": \"Insert site-specific NAI here\",
            \"Action\": \"Cross-reference source IP addresses from remote service logs (Zeek conn.log, VPN/Citrix logs) against a list of IP addresses identified in other security alerts or known compromised host lists. Inner join connection logs with Threat Intelligence feeds for known malicious IPs connecting via remote services.\"
          }
        }
      }
    }
  }
]"""),
                # --- END OF MODEL EXAMPLE JSON OUTPUT ---
            ],
        ),
        # Turn 3: User provides the actual prompt containing the target technique details.
        types.Content(
            role="user",
            parts=[
                # The 'prompt' variable passed into the function contains the technique details
                # for which the ASOM should be generated this time.
                types.Part.from_text(text=prompt),
            ],
        ),
    ]

    # Configure the generation request.
    generate_content_config = types.GenerateContentConfig(
        # Specify that the model's response should be formatted as JSON.
        response_mime_type="application/json",
        # Provide system instructions, reinforcing the desired JSON structure.
        # This repeats the template definition from the initial user message for emphasis.
        system_instruction=[
            # --- START OF SYSTEM INSTRUCTION JSON TEMPLATE (DO NOT MODIFY CONTENT) ---
            types.Part.from_text(text="""Based on these definitions, please generate a detailed ASOM in the JSON format. The keys in the JSON object should correspond to the following ASOM components in this order: IR, Indicators, Evidence, Data Platform, Data, NAIs, Actions. Note that the key for IR should be replaced by the actual IR based on the description above. Also note that the key for evidence should be replaced with the actual form of evidence based on the description above. Here is an example template:

{
    \"(Insert IR here)\": {
        \"Indicators\": {
            \"(Insert form of evidence here)\": {
                \"Data\": \"\",
                \"Data Platform\": \"\",
                \"NAI\": \"\",
                \"Action\": \"\"
            },
            \"(Insert form of evidence here)\": {
                \"Data\": \"\",
                \"Data Platform\": \"\",
                \"NAI\": \"\",
                \"Action\": \"\"
            }
        }
    }
}"""),
            # --- END OF SYSTEM INSTRUCTION JSON TEMPLATE ---
        ],
    )

    # Send the request to the generative model.
    response = client.models.generate_content(
        model=model,              # The specified target model
        contents=contents,        # The constructed multi-turn conversation history
        config=generate_content_config, # Configuration including response format and system instructions
    )

    # Return the text content of the model's response, which should be the generated ASOM JSON.
    return response.text

Since we will go through multiple iterations of generating and refining these analytic plans, we'll create a wrapper function `prompt_model` that makes it easy to map a collection of prompts to a specific generator function.

In [ ]:
def prompt_model(requests_per_minute: int,
                 max_iterations: int,
                 prompts_collection: dict,
                 generation_function,
                 output_directory: str = ".",
                 model_name: str = "gemini-2.5-flash-preview-04-17"): # Default model, can be overridden
    """
    Processes a collection of prompts using a specified generation function,
    while respecting rate limits and iteration caps.

    Args:
        requests_per_minute: The target maximum number of requests allowed per minute.
        max_iterations: The maximum number of successful generation requests to perform.
        prompts_collection: A dictionary where keys are identifiers (e.g., technique names)
                            and values are the prompts themselves.
        generation_function: A function reference that takes a prompt and a model_name
                             and returns the generated content.
                             Example: def my_generator(prompt, target_model): ... return response
        output_directory: The directory where output files will be saved. Defaults to current dir.
        model_name: The name or identifier of the model to be used by the generation_function.
    """
    if requests_per_minute <= 0:
        min_seconds_between_requests = 0
    else:
        min_seconds_between_requests = 60 / requests_per_minute

    last_request_timestamp = 0  # Timestamp of the last request start
    successful_requests_count = 0 # Counter for successful requests

    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        log(f"Created output directory: {output_directory}")

    log(f"Starting prompt processing. Max iterations: {max_iterations}, RPM: {requests_per_minute}.")

    for item_key, prompt_content in prompts_collection.items():
        # Construct the expected output filename based on the item key.
        # Ensures that the filename is safe for filesystems.
        safe_item_key = "".join(c if c.isalnum() or c in (' ', '.', '_') else '_' for c in str(item_key)).rstrip()
        output_filename = f"{safe_item_key}.json" # Assuming JSON output, adjust as needed
        output_file_path = os.path.join(output_directory, output_filename)

        # Check if a file with this name already exists in the output directory.
        if os.path.exists(output_file_path):
            log(f"Skipping '{item_key}': {output_file_path} already exists.")
            continue  # Skip to the next item

        # Rate limiting check
        if min_seconds_between_requests > 0:
            current_time = time.time()
            elapsed_time_since_last_request = current_time - last_request_timestamp

            if elapsed_time_since_last_request < min_seconds_between_requests:
                wait_time = min_seconds_between_requests - elapsed_time_since_last_request
                log(f"Rate limit check: Waiting for {wait_time:.2f} seconds...")
                time.sleep(wait_time)

        log(f"Processing item #{successful_requests_count + 1}/{max_iterations}). Generating content for '{item_key}'...")

        # Record the time just before starting the generative API call.
        last_request_timestamp = time.time()
        try:
            # Call the external function to generate content
            response_content = generation_function(prompt_content, target_model=model_name)

            # Increment the successful request counter *only after* the API call
            # returns without raising an exception.
            successful_requests_count += 1

            # Open the target output file in write mode ('w').
            with open(output_file_path, "w") as f:
                f.write(response_content) # Assumes response_content is a string

            log(f"Successfully generated content for '{item_key}' and saved as {output_file_path}")

        except Exception as e:
            log(f"Error generating content for '{item_key}': {e}")
            # Option to stop the entire loop: break
            # Option to continue with the next item: continue
            # Current implementation stops on error:
            log("Stopping processing due to an error.")
            break

        # Check if the number of *successful requests* has reached the set limit.
        if successful_requests_count >= max_iterations:
            log(f"Generation limit of {max_iterations} successful requests reached. Exiting.")
            break

    log("Prompt processing finished.")

Next, generate the analytic plans.

In [ ]:
prompt_model(
    requests_per_minute = 10,
    max_iterations = 3,
    prompts_collection = prompt_library,
    generation_function = generate_initial_plan,
    output_directory = "./",
    model_name = "gemini-2.5-flash-preview-04-17"
)

In [ ]:
# # Defines the target maximum number of requests allowed per minute.
# REQUESTS_PER_MINUTE = 10

# # Calculates the minimum required time separation (in seconds) between the start
# # of consecutive requests to stay within the defined rate limit.
# MIN_SECONDS_BETWEEN_REQUESTS = 60 / REQUESTS_PER_MINUTE

# # Stores the timestamp (seconds since epoch) of when the last request started.
# # Initialized to 0 to ensure the very first request in the script's run
# # isn't unnecessarily delayed by the rate-limiting check.
# last_request_time = 0
# # Counts the number of successful requests made within this specific run
# # of the script. This is used to enforce the `limit`.
# request_count_in_loop = 0

# # Sets a maximum limit on the number of successful generation requests
# # (`generate_initial_plan` calls that don't raise an immediate exception)
# # that this script run will perform. This is set to 500 due to API request
# # limits for the Gemini free tier.
# limit = 500

# # Iterate through each technique name (key) in the prompt library.
# for technique in prompt_library:
#     # Construct the expected output filename based on the technique name.
#     output_filename = f"{technique}.json"
#     # Check if a file with this name already exists in the current directory.
#     if os.path.exists(output_filename):
#         # If the file exists, log a message indicating it's being skipped.
#         log(f"Skipping {technique}: {output_filename} already exists.")
#         continue # Skip to the next technique

#     # Get the current time before potentially making a request.
#     current_time = time.time()
#     # Calculate the time elapsed since the start of the last request.
#     elapsed_time = current_time - last_request_time

#     # Check if the elapsed time is less than the minimum required separation.
#     if elapsed_time < MIN_SECONDS_BETWEEN_REQUESTS:
#         # If too soon, calculate the necessary waiting time.
#         wait_time = MIN_SECONDS_BETWEEN_REQUESTS - elapsed_time
#         # Log that the script is pausing for rate limiting.
#         log(f"Rate limit check: Waiting for {wait_time:.2f} seconds...")
#         # Pause the script's execution for the calculated duration.
#         time.sleep(wait_time)

#     # Log the attempt to process the current technique, showing its progress
#     # relative to the `limit` for this run.
#     log(f"Processing #{request_count_in_loop + 1}/{limit}). Started generating plan for {technique} ...")

#     # Record the time just before starting the generative API call.
#     # This ensures the rate limit is based on request initiation frequency.
#     last_request_time = time.time()
#     try:
#         # Call the external function to generate the plan using the prompt
#         # associated with the current technique and the specified model.
#         response = generate_initial_plan(prompt_library[technique], target_model = "gemini-2.5-flash-preview-04-17") # Example model

#         # Increment the successful request counter *only after* the API call
#         # returns without raising an exception.
#         request_count_in_loop += 1

#         # Open the target output file in write mode ('w').
#         with open(output_filename, "w") as f:
#             # Write the content received from the `generate_initial_plan` function to the file.
#             f.write(response)
        
#         # Log the successful generation and saving of the plan.
#         log(f"Successfully generated plan for '{technique}' and saved as {output_filename}")

#     except Exception as e:
#         # If any exception occurs during the `generate_initial_plan` call:
#         # Log the error message, including the technique that failed.
#         log(f"Error generating plan for {technique}: {e}")
#         # Stop the entire loop immediately upon encountering an error.
#         # Alternative strategies could be to `continue` (skip the failed technique)
#         # or implement retry logic.
#         break # Stop processing further techniques on error

#     # Check if the number of *successful requests* made in this run has reached the set limit.
#     if request_count_in_loop >= limit:
#         # If the limit is reached, log a message explaining why the loop is ending.
#         log(f"Generation limit of {limit} reached. Exiting.")
#         # Stop the loop.
#         break

# # Log a final message indicating the main processing loop has finished, either by
# # completing all techniques, hitting the generation limit, or encountering an error.
# log("Processing finished.")

## Build Prompts to Improve the Analytic Plans

This section generates a prompt to build the initial analytic plan for each technique in the MITRE ATT&CK matrix. It first defines a base prompt that provides context to the model, and then generates a specific prompt crafted for each individual technique. The complete prompts are stored in `prompt_library`, where the key is the technique ID and name in the form "T1190 - Exploit Public-Facing Application".

In [ ]:
base_prompt = """\

"""

In [ ]:
refine_prompt_library = {}

for technique in technique_dict:
    refine_prompt_library[technique] = base_prompt[0] + f"\n\nTechnique: {technique}\n\nTactic(s): {technique_dict[technique]['tactic']}\n\nDescription: {technique_dict[technique]['description']}\n\nDetection: {technique_dict[technique]['detection']}\n\n"

## Improve the Analytic Plans

This section iterates over the existing analytic plans, then uses generative artificial intelligence to improve them.

First, define a function with embeded context for improving the analytic plans. This function is unique to the improvement of analytic plans.

In [ ]:
def generate_improved_plan(prompt, target_model):
    """Generates an improved analytic plan in JSON format using a generative AI model.

    This function constructs a detailed, multi-turn prompt including definitions, examples,
    and constraints to guide the target generative model in creating an analytic plan tailored
    to a specific cyber threat scenario and MITRE ATT&CK technique. It leverages
    few-shot prompting by providing a complete example (for T1133) within the prompt context.

    Args:
        prompt (str): A string containing the specific details of the MITRE ATT&CK
                      technique for which the ASOM should be generated. This typically
                      includes Technique ID, Name, Tactics, Description, and Detection guidance.
        target_model (str): The name or identifier of the generative AI model to use
                            (e.g., "models/gemini-1.5-flash").

    Returns:
        str: A string containing the generated ASOM in JSON format. Returns the raw text
             response from the model. Error handling for API calls or invalid JSON
             responses is not explicitly included here.
    """
    return None

Next, generate the improved analytic plans.

In [ ]:
prompt_model(
    requests_per_minute = 10,
    max_iterations = 3,
    prompts_collection = refine_prompt_library,
    generation_function = generate_improved_plan,
    output_directory = "./",
    model_name = "gemini-2.5-flash-preview-04-17"
)

In [ ]:
# Halt execution before the code below is run
sys.exit(0)

In [ ]:
def update_json_files_structure(root_dir, dict_to_add_or_update):
    """
    Recursively finds JSON files in root_dir and adds or updates keys
    from dict_to_add_or_update into the dictionary value of the primary
    key within each item of the main list.

    Args:
        root_dir (str): The path to the directory to start searching from.
        dict_to_add_or_update (dict): A dictionary containing key-value pairs
                                      to add or update.
    """
    if not isinstance(dict_to_add_or_update, dict):
        log("Error: 'dict_to_add_or_update' must be a dictionary.")
        return

    if not dict_to_add_or_update:
        log("Warning: 'dict_to_add_or_update' is empty. No changes will be made.")
        # Optionally return here if desired, or proceed to scan files anyway.
        # return

    if not os.path.isdir(root_dir):
        log(f"Error: Directory '{root_dir}' not found.")
        return

    log(f"Starting scan in directory: {root_dir}")
    log(f"Data to add/update: {dict_to_add_or_update}")
    files_processed = 0
    files_modified = 0
    errors_encountered = 0
    error_files = []
    warning_files = []

    for subdir, _, files in os.walk(root_dir):
        for filename in files:
            if filename.lower().endswith('.json'):
                file_path = os.path.join(subdir, filename)
                log(f"Processing file: {file_path}")
                files_processed += 1
                modified_in_this_file = False
                try:
                    # Read the JSON file with UTF-8 encoding
                    # Use a variable for the file handle to ensure it's closed
                    # even if errors occur before json.load() finishes
                    read_f = None
                    try:
                        read_f = open(file_path, 'r', encoding='utf-8')
                        data = json.load(read_f)
                    finally:
                        if read_f:
                            read_f.close()


                    # Ensure the top level is a list
                    if isinstance(data, list):
                        # Iterate through each item (dictionary) in the list
                        for item in data:
                            if isinstance(item, dict):
                                # Iterate through the primary keys in the item dictionary
                                # (Assuming one primary key per item as per example)
                                for primary_key in list(item.keys()): # Use list() for safe iteration
                                    # Check if the value associated with the primary key is a dictionary
                                    if isinstance(item[primary_key], dict):
                                        target_dict = item[primary_key]
                                        # Iterate through the keys and values to add/update
                                        for key_to_update, value_to_update in dict_to_add_or_update.items():
                                            # Check if the key needs to be added or if the value is different
                                            if key_to_update not in target_dict or target_dict[key_to_update] != value_to_update:
                                                target_dict[key_to_update] = value_to_update
                                                modified_in_this_file = True # Mark that a change occurred
                                    else:
                                        log(f"  Warning: Value for key '{primary_key}' in {file_path} is not a dictionary. Skipping update for this key.")
                                        warning_files.append(file_path)
                            else:
                                 log(f"  Warning: Found an item in the list that is not a dictionary in {file_path}. Skipping this item.")
                                 warning_files.append(file_path)

                    else:
                        log(f"  Warning: Root object in {file_path} is not a list. Skipping this file.")
                        warning_files.append(file_path)
                        continue # Skip to the next file

                    # Write the modified data back to the file if changes were made
                    if modified_in_this_file:
                        write_f = None
                        try:
                            write_f = open(file_path, 'w', encoding='utf-8')
                            # Use indent for pretty printing, adjust as needed (e.g., indent=4)
                            json.dump(data, write_f, indent=2, ensure_ascii=False)
                            log(f"  Successfully updated: {file_path}")
                            files_modified += 1
                        finally:
                             if write_f:
                                 write_f.close()
                    else:
                        log(f"  No changes needed for: {file_path}")


                except json.JSONDecodeError:
                    log(f"  Error: Invalid JSON format in {file_path}. Skipping.")
                    error_files.append(file_path)
                    errors_encountered += 1
                except IOError as e:
                    log(f"  Error: Could not read/write file {file_path}. Reason: {e}. Skipping.")
                    error_files.append(file_path)
                    errors_encountered += 1
                except Exception as e:
                    # Log the type of exception for better debugging
                    log(f"  Error: An unexpected {type(e).__name__} occurred processing {file_path}. Reason: {e}. Skipping.")
                    error_files.append(file_path)
                    errors_encountered += 1

    print("\n--- Processing Summary ---")
    print(f"Total files scanned: {files_processed}")
    print(f"Files successfully modified: {files_modified}")
    # Corrected calculation for skipped/unchanged files
    skipped_unchanged = files_processed - files_modified - errors_encountered
    print(f"Files skipped, unchanged, or with warnings: {skipped_unchanged}")
    print(f"Errors encountered during processing: {errors_encountered}")
    print("--------------------------")
    print("Error files:\n",error_files)
    print()
    print("Warning files:\n",warning_files)

In [ ]:
# target_directory = "./"

# data_to_add = {
#     "version": "1.0",
#     "last_updated": "2025-05-04"
# }

# # Call the generalized function
# update_json_files_structure(target_directory, data_to_add)